## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather description
0,0,Dikson,RU,73.5069,80.5464,5.79,89,100,10.02,overcast clouds
1,1,Ca Mau,VN,9.1769,105.1500,75.45,91,99,7.07,overcast clouds
2,2,Katsuura,JP,35.1333,140.3000,46.51,68,0,9.66,clear sky
3,3,Mahebourg,MU,-20.4081,57.7000,76.24,88,40,4.61,scattered clouds
4,4,Lebu,CL,-37.6167,-73.6500,60.15,75,22,8.57,few clouds
5,5,San Cristobal,VE,7.7669,-72.2250,68.18,91,69,2.59,broken clouds
6,6,Albany,US,42.6001,-73.9662,40.75,52,3,3.51,clear sky
7,7,Karratha,AU,-20.7377,116.8463,80.51,47,74,9.66,broken clouds
8,8,Rikitea,PF,-23.1203,-134.9692,76.59,78,18,15.12,few clouds
9,9,Tabou,CI,4.4230,-7.3528,80.08,83,38,6.87,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
temp_city_data_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
temp_city_data_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather description
1,1,Ca Mau,VN,9.1769,105.1500,75.45,91,99,7.07,overcast clouds
3,3,Mahebourg,MU,-20.4081,57.7000,76.24,88,40,4.61,scattered clouds
7,7,Karratha,AU,-20.7377,116.8463,80.51,47,74,9.66,broken clouds
8,8,Rikitea,PF,-23.1203,-134.9692,76.59,78,18,15.12,few clouds
9,9,Tabou,CI,4.4230,-7.3528,80.08,83,38,6.87,scattered clouds
...,...,...,...,...,...,...,...,...,...,...
664,664,Becerril,CO,9.7041,-73.2793,80.64,89,96,0.67,overcast clouds
666,666,Chake Chake,TZ,-5.2459,39.7666,77.83,90,64,6.93,broken clouds
678,678,Raga,SS,8.4596,25.6780,75.56,25,1,4.70,clear sky
684,684,Humaita,BR,-7.5061,-63.0208,84.65,70,100,2.91,light rain


In [5]:
# 4a. Determine if there are any empty rows.
temp_city_data_df.isnull().sum()

City_ID                0
City                   0
Country                4
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Weather description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = temp_city_data_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Weather description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather description,Lat,Lng,Hotel Name
1,Ca Mau,VN,75.45,overcast clouds,9.1769,105.1500,
3,Mahebourg,MU,76.24,scattered clouds,-20.4081,57.7000,
7,Karratha,AU,80.51,broken clouds,-20.7377,116.8463,
8,Rikitea,PF,76.59,few clouds,-23.1203,-134.9692,
9,Tabou,CI,80.08,scattered clouds,4.4230,-7.3528,
10,Georgetown,MY,76.98,thunderstorm with light rain,5.4112,100.3354,
21,Kasongo,CD,70.02,light rain,-4.4500,26.6667,
27,Sal Rei,CV,77.56,overcast clouds,16.1833,-22.9167,
32,Vaini,TO,84.36,few clouds,-21.2000,-175.2000,
33,Kawalu,ID,72.99,overcast clouds,-7.3817,108.2082,


In [12]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [15]:
hotel_df.head(10)

,City,Country,Max Temp,Weather description,Lat,Lng,Hotel Name
1,Ca Mau,VN,75.45,overcast clouds,9.1769,105.1500,Khách Sạn Mường Thanh Luxury Cà Mau
3,Mahebourg,MU,76.24,scattered clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
7,Karratha,AU,80.51,broken clouds,-20.7377,116.8463,ibis Styles Karratha
8,Rikitea,PF,76.59,few clouds,-23.1203,-134.9692,People ThankYou
9,Tabou,CI,80.08,scattered clouds,4.4230,-7.3528,hotêl doufoulougou
10,Georgetown,MY,76.98,thunderstorm with light rain,5.4112,100.3354,Cititel Penang
21,Kasongo,CD,70.02,light rain,-4.4500,26.6667,Guest Hôtel
27,Sal Rei,CV,77.56,overcast clouds,16.1833,-22.9167,Marine Club Beach Resort
32,Vaini,TO,84.36,few clouds,-21.2000,-175.2000,Keleti Beach Resort
33,Kawalu,ID,72.99,overcast clouds,-7.3817,108.2082,KOSAN UNGU UNSIK


In [16]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"]!="")]

In [17]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [22]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather description</dt><dd>{Weather description}</dd>
<dt>Max Temp</dt><dd>{Max Temp}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [23]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))